In [ ]:
pip install category_encoders

In [ ]:
# importa o pandas
import pandas as pd

# gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Escalonamento multidmensional
from sklearn.manifold import MDS

# transformação de atributos
from category_encoders import OneHotEncoder, OrdinalEncoder

# cálculo de distâncias
from scipy.spatial.distance import pdist, squareform

In [3]:
base = pd.read_csv("https://raw.githubusercontent.com/jlyang1990/Spark_Python_Do_Big_Data_Analytics/master/auto-data.csv")

In [4]:
base.sample(10)

,MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE
175,porsche,gas,std,two,hatchback,rwd,four,143,5500,19,27,22018
6,nissan,gas,std,two,sedan,fwd,four,69,5200,31,37,5499
2,mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195
138,peugot,gas,std,four,sedan,rwd,four,95,5000,19,24,15580
92,toyota,gas,std,four,hatchback,fwd,four,92,4200,27,32,9988
61,subaru,gas,std,four,wagon,4wd,four,82,4800,23,29,8013
36,toyota,gas,std,four,hatchback,fwd,four,70,4800,30,37,7198
34,subaru,gas,std,four,sedan,fwd,four,82,4800,32,37,7126
54,toyota,gas,std,four,wagon,4wd,four,62,4800,27,32,7898
42,subaru,gas,std,four,wagon,fwd,four,82,4800,28,32,7463


In [5]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAKE       197 non-null    object
 1   FUELTYPE   197 non-null    object
 2   ASPIRE     197 non-null    object
 3   DOORS      197 non-null    object
 4   BODY       197 non-null    object
 5   DRIVE      197 non-null    object
 6   CYLINDERS  197 non-null    object
 7   HP         197 non-null    int64 
 8   RPM        197 non-null    int64 
 9   MPG-CITY   197 non-null    int64 
 10  MPG-HWY    197 non-null    int64 
 11  PRICE      197 non-null    int64 
dtypes: int64(5), object(7)
memory usage: 18.6+ KB


In [6]:
#descrição dos atributos numéricos
base.describe().T

,count,mean,std,min,25%,50%,75%,max
HP,197.0,103.604061,37.639205,48.0,70.0,95.0,116.0,262.0
RPM,197.0,5118.020305,481.035914,4150.0,4800.0,5200.0,5500.0,6600.0
MPG-CITY,197.0,25.152284,6.437863,13.0,19.0,24.0,30.0,49.0
MPG-HWY,197.0,30.629442,6.836259,16.0,25.0,30.0,34.0,54.0
PRICE,197.0,13279.644670,8010.334218,5118.0,7775.0,10345.0,16503.0,45400.0


In [7]:
#correlação entre os atributos numéricos
base_corr = base.corr()
base_corr

,HP,RPM,MPG-CITY,MPG-HWY,PRICE
HP,1.000000,0.103574,-0.821960,-0.803658,0.811953
RPM,0.103574,1.000000,-0.107648,-0.050632,-0.101319
MPG-CITY,-0.821960,-0.107648,1.000000,0.972407,-0.692948
MPG-HWY,-0.803658,-0.050632,0.972407,1.000000,-0.708659
PRICE,0.811953,-0.101319,-0.692948,-0.708659,1.000000


In [8]:
top5 = base_corr.sum(axis=1).sort_values(ascending=False).head(5)
top5

RPM         0.843974
MPG-HWY     0.409458
MPG-CITY    0.349851
PRICE       0.309027
HP          0.289909
dtype: float64

In [9]:
base.describe(include = 'object').T

,count,unique,top,freq
MAKE,197,21,toyota,32
FUELTYPE,197,2,gas,178
ASPIRE,197,2,std,162
DOORS,197,2,four,112
BODY,197,5,sedan,92
DRIVE,197,3,fwd,114
CYLINDERS,197,7,four,153


In [10]:
#categórico x categórico
pd.crosstab(base['FUELTYPE'],base['DOORS'])

DOORS,four,two
FUELTYPE,,
diesel,16,3
gas,96,82


In [11]:
#categórico x numérico
base.groupby('BODY')['PRICE'].max()

BODY
convertible    37028
hardtop        45400
hatchback      22018
sedan          41315
wagon          28248
Name: PRICE, dtype: int64

In [12]:
#categórico x numérico
base.groupby('DRIVE')['MPG-CITY'].describe()

,count,mean,std,min,25%,50%,75%,max
DRIVE,,,,,,,,
4wd,8.0,24.000000,2.927700,18.0,23.0,24.0,26.25,27.0
fwd,114.0,28.210526,6.126215,17.0,24.0,28.0,31.00,49.0
rwd,75.0,20.626667,4.042923,13.0,17.0,19.0,24.00,31.0


In [13]:
#categórico x numérico
base.groupby('DRIVE')['MPG-CITY'].max()

DRIVE
4wd    27
fwd    49
rwd    31
Name: MPG-CITY, dtype: int64

In [14]:
base2 = pd.read_csv("simpson.csv")

In [15]:
base2
base2.sample(10)

,x,y,a1,a2,a3,a4
10,6.071324,9.339657,v3,v3,v2,v4
107,9.552233,5.546016,v1,v2,v2,v2
151,13.248856,8.395817,v2,v3,v3,v4
276,1.938686,-2.047043,v2,v2,v1,v1
299,5.509466,4.066866,v3,v2,v2,v4
25,3.686403,0.645317,v3,v3,v1,v1
266,0.216485,2.072074,v2,v3,v1,v1
185,14.678849,10.233663,v1,v2,v3,v3
114,4.253815,0.227375,v1,v1,v1,v3
41,10.381660,6.587870,v1,v2,v3,v4


In [16]:
base2.describe(include = 'object').T

,count,unique,top,freq
a1,300,3,v1,100
a2,300,3,v1,100
a3,300,3,v1,100
a4,300,4,v1,75


In [17]:
for column_name in base2.select_dtypes(include=["object_"]):
  print(column_name, "->", base2[column_name].unique())
  print()

a1 -> ['v2' 'v3' 'v1']

a2 -> ['v1' 'v3' 'v2']

a3 -> ['v3' 'v1' 'v2']

a4 -> ['v3' 'v4' 'v1' 'v2']



In [18]:
categorical = ['a1', 'a2', 'a3', 'a4']

In [19]:
onehotenc = OneHotEncoder(cols = categorical)

In [ ]:
base3 = onehotenc.fit_transform(base2)

In [21]:
base3.head()

,x,y,a1_1,a1_2,a1_3,a2_1,a2_2,a2_3,a3_1,a3_2,a3_3,a4_1,a4_2,a4_3,a4_4
0,13.944627,8.130532,1,0,0,1,0,0,1,0,0,1,0,0,0
1,2.781110,-1.199000,1,0,0,0,1,0,0,1,0,0,1,0,0
2,11.558056,8.062959,1,0,0,0,0,1,1,0,0,0,1,0,0
3,1.452319,-3.691675,1,0,0,1,0,0,0,1,0,0,1,0,0
4,10.862521,5.505988,0,1,0,1,0,0,1,0,0,1,0,0,0


In [22]:
correlacao = base2.corr()

In [23]:
correlacao

,x,y
x,1.000000,0.796013
y,0.796013,1.000000


In [24]:
#correlacao entre todos os atributos da base
correlacao2 = base3.corr()
correlacao2

,x,y,a1_1,a1_2,a1_3,a2_1,a2_2,a2_3,a3_1,a3_2,a3_3,a4_1,a4_2,a4_3,a4_4
x,1.000000,0.796013,-2.362399e-02,6.183755e-02,-0.038214,6.474866e-03,0.097039,-1.035137e-01,0.803472,-0.828577,0.025105,-0.076615,-6.034580e-02,6.876085e-02,6.820011e-02
y,0.796013,1.000000,-2.513004e-02,5.236502e-02,-0.027235,-3.538882e-02,0.105276,-6.988670e-02,0.775073,-0.714976,-0.060097,-0.066886,-7.384812e-02,6.033370e-02,8.040090e-02
a1_1,-0.023624,-0.025130,1.000000e+00,-5.000000e-01,-0.500000,4.000000e-02,0.010000,-5.000000e-02,-0.020000,0.040000,-0.020000,-0.032660,1.269091e-17,3.265986e-02,8.385063e-18
a1_2,0.061838,0.052365,-5.000000e-01,1.000000e+00,-0.500000,-5.000000e-02,0.055000,-5.000000e-03,0.025000,-0.095000,0.070000,0.081650,-3.265986e-02,-4.532467e-19,-4.898979e-02
a1_3,-0.038214,-0.027235,-5.000000e-01,-5.000000e-01,1.000000,1.000000e-02,-0.065000,5.500000e-02,-0.005000,0.055000,-0.050000,-0.048990,3.265986e-02,-3.265986e-02,4.898979e-02
a2_1,0.006475,-0.035389,4.000000e-02,-5.000000e-02,0.010000,1.000000e+00,-0.500000,-5.000000e-01,-0.005000,-0.005000,0.010000,0.016330,4.169869e-17,-1.632993e-02,-1.359740e-17
a2_2,0.097039,0.105276,1.000000e-02,5.500000e-02,-0.065000,-5.000000e-01,1.000000,-5.000000e-01,0.070000,-0.110000,0.040000,0.081650,-6.531973e-02,1.632993e-02,-3.265986e-02
a2_3,-0.103514,-0.069887,-5.000000e-02,-5.000000e-03,0.055000,-5.000000e-01,-0.500000,1.000000e+00,-0.065000,0.115000,-0.050000,-0.097980,6.531973e-02,-1.269091e-17,3.265986e-02
a3_1,0.803472,0.775073,-2.000000e-02,2.500000e-02,-0.005000,-5.000000e-03,0.070000,-6.500000e-02,1.000000,-0.500000,-0.500000,-0.114310,1.632993e-02,4.898979e-02,4.898979e-02
a3_2,-0.828577,-0.714976,4.000000e-02,-9.500000e-02,0.055000,-5.000000e-03,-0.110000,1.150000e-01,-0.500000,1.000000,-0.500000,0.016330,8.164966e-02,-6.531973e-02,-3.265986e-02


In [25]:
agrupado = base2.groupby('a1').sum()
agrupado

,x,y
a1,,
v1,721.813305,337.423335
v2,730.696707,338.670312
v3,782.733141,384.578594


In [26]:
agrupado_a1 = base2.groupby('a1').sum()
agrupado_a1

,x,y
a1,,
v1,721.813305,337.423335
v2,730.696707,338.670312
v3,782.733141,384.578594


In [27]:
agrupado_a2 = base2.groupby('a2').sum()
agrupado_a2

,x,y
a2,,
v1,749.023515,332.592983
v2,682.052856,312.156340
v3,804.166783,415.922918


In [28]:
agrupado_a3 = base2.groupby('a3').sum()
agrupado_a3

,x,y
a3,,
v1,240.571033,-69.996277
v2,760.367230,317.955634
v3,1234.304891,812.712884


In [29]:
agrupado_a4 = base2.groupby('a4').sum()
agrupado_a4

,x,y
a4,,
v1,597.268636,297.999044
v2,596.955016,308.918746
v3,515.960035,228.771335
v4,525.059467,224.983115
